This notebook is to process New York Data

In [5]:
import numpy as np

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json

#pip install geopy
from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

import folium

print('Libraries Imported!')

Libraries Imported!


In [6]:
# download newyork data
!pip install wget
import wget
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Defaulting to user installation because normal site-packages is not writeable
Data downloaded!


'wget' is not recognized as an internal or external command,
operable program or batch file.


In [7]:
with open(r'C:\NecessaryFolders\PYTHON_IBM_DATA_SCIENCE\Capstone_Project\Final_Project\newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Observe one data collection to understand the structure of json

In [11]:
NY_neighborhoods_data = newyork_data['features']

NY_neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

In [14]:
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude']

NY_neighborhoods = pd.DataFrame(columns = column_names)

NY_neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [15]:
for data in NY_neighborhoods_data:
    borough = data['properties']['borough']
    neighborhood_name = data['properties']['name']
    
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    NY_neighborhoods = NY_neighborhoods.append({'Borough': borough,
                                               'Neighborhood': neighborhood_name,
                                               'Latitude': neighborhood_lat,
                                               'Longitude': neighborhood_lon}, ignore_index = True)

Store the data

In [17]:
NY_neighborhoods.head()
NY_neighborhoods.to_excel(r'C:\NecessaryFolders\PYTHON_IBM_DATA_SCIENCE\Capstone_Project\Final_Project\submission\NY_data.xlsx', index = None, header=True, encoding = 'utf8-8')

print('The Dataframe has {} boroughs and {} neighborhoods.'.format(
        len(NY_neighborhoods['Borough'].unique()),
        NY_neighborhoods.shape[0]
))

The Dataframe has 5 boroughs and 306 neighborhoods.


In [18]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent = 'ny_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude

print('The geographical coordinate of New York City are {} {}.'.format(latitude, longitude))

The geographical coordinate of New York City are 40.7127281 -74.0060152.


In [19]:
map_newyork = folium.Map(location = [latitude, longitude], zoom_start = 10)

In [20]:
for lat, lng, borough, neighborhood in zip(NY_neighborhoods['Latitude'], NY_neighborhoods['Longitude'], NY_neighborhoods['Borough'], NY_neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_newyork)

map_newyork

In [49]:
CLIENT_ID = 'P0CICFIWIUCIGNJ3INWISZ4YWZJGXT12DLAIAWJRKD0V0KOE' # your Foursquare ID
CLIENT_SECRET = 'WPOXXNW4QCLQEKKZM1OYC4MAQOXKZF2GDUG5M1Q0YXQ13ZAW' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: P0CICFIWIUCIGNJ3INWISZ4YWZJGXT12DLAIAWJRKD0V0KOE
CLIENT_SECRET:WPOXXNW4QCLQEKKZM1OYC4MAQOXKZF2GDUG5M1Q0YXQ13ZAW


In [50]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [51]:
NY_venues = getNearbyVenues(names = NY_neighborhoods['Neighborhood'], latitudes = NY_neighborhoods['Latitude'], longitudes = NY_neighborhoods['Longitude'])

Wakefield


KeyError: 'groups'

In [33]:
print(NY_venues.shape)
NY_venues.head()

NameError: name 'NY_venues' is not defined

In [ ]:
NY_venues.groupby('Neighborhood').count()

In [25]:
NY_onehot = pd.get_dummies(NY_venues[['Venue Category']], prefix = "", prefix_sep = "")

NY_onehot['Neighborhood'] = NY_venues['Neighborhood']

col_list = list(NY_onehot.columns)
col_list.remove('Neighborhood')
cols = ['Neighborhood'] + [col for col in col_list]

NY_onehot = NY_onehot[cols]

NY_onehot.head()

NameError: name 'NY_venues' is not defined

In [52]:
NY_grouped = NY_onehot.groupby('Neighborhood').mean().reset_index()
NY_grouped

NameError: name 'NY_onehot' is not defined

In [53]:
NY_grouped.shape

NameError: name 'NY_grouped' is not defined

In [54]:
export_excel = NY_grouped.to_excel(r'C:\NecessaryFolders\PYTHON_IBM_DATA_SCIENCE\Capstone_Project\Final_Project\submission\NY_grouped.xlsx', index = None, header=True, encoding = 'utf8-8')

NameError: name 'NY_grouped' is not defined